In [ ]:
def PHILD( x, y, x1, y1, x2, y2 ):
    ## transform from global coordinates to panel coordinates (coordinates)
    xp, yp, xp1, yp1, xp2, yp2, alpha = GlobalToPanelCoords( x, y, x1, y1, x2, y2 )
    
    ## compute induced velocity potential by the doublet element (in panel coordinates)
    th1 = atan2( yp, xp-xp1 ); th2 = atan2( yp, xp-xp2 )
    r1 = (xp-xp1)**2 + yp**2; r2 = (xp-xp2)**2 + yp**2
    d = (xp2 - xp1)
    
    phi_a = -( th2 - th1 )/2/pi \
    + xp*(th2 - th1)/d/2/pi \
    + yp * log( r2/r1 )/4/pi/d
    
    phi_b = -xp*( th2 - th1 )/2/pi/d \
    - yp*log(r2/r1)/4/pi/d
    
    
    phi_a = -( th2 - th1 )/2/pi \
    + ( 2*xp*( th2 - th1 ) + yp * log( r2/r1 ) )/4/pi/(xp2-xp1)
    phi_b =  -( 2*xp*( th2 - th1 ) + yp * log( r2/r1 ) )/4/pi/(xp2-xp1)

    
    return phi_a, phi_b

def DirichletLinearStrengthDoubletMethod( X, Y, N, U0, AOA ):
    
    # generate geometry
    # for N panels we need N+1 points
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tck = createSplines( X, Y )
    x, y = computeNodeLocations( N+1, tck )
#     x = x[::-1]; y[::-1]
    dx = np.diff(x)
    dy = np.diff(y)
    panelLength = np.sqrt( dx**2 + dy**2 )

    ## generate collocation point data
    xc = ( x[1:] + x[:-1] )/2; yc = ( y[1:] + y[:-1] )/2;
    panelAlpha = np.arctan2( dy, dx )
    nxc = np.sin( panelAlpha )
    nyc = -np.cos( panelAlpha )
    
    ## displace collocation point inward by 0.05 of the panel length
    xc = xc - 0.05*panelLength*nxc; yc = yc - 0.05*panelLength*nyc;
    
    ## generate wake geometry
    tckw = createWakeGeometry( X, Y, AOA )
    xw, yw = computeNodeLocations( N+1, tckw )
    
    # generate a (N+2)x(N+2) system of equations
    # where the last row is to apply the kutta-condition
    A = np.zeros( (N+2, N+2) )
    b = np.zeros( N+2 )
    
    ## row of matrix
    for i in range( N ):
        b[i] = -U0 * ( xc[i] * np.cos( AOA * pi / 180 ) + yc[i] * np.sin( AOA * pi / 180 ) )
        for j in range( N ):
            phi_a, phi_b = PHILD( xc[i], yc[i], x[j], y[j], x[j+1], y[j+1] )
            A[i, j] = A[i, j] + phi_a
            A[i, j+1] = A[i, j+1] + phi_b

            
            
    ## include wake effect
    for i in range( N ):
        phi = PHICD( xc[i], yc[i], xw[0], yw[0], xw[-1], yw[-1] )
        A[i, -1] = A[i, -1] + phi
            
    ## include Kutta Condition
    A[N, 0] = 1; A[N, -2] = -1; A[N, -1] = 1  
    A[N+1, 0] = -1; A[N+1, 1] = 1; A[N+1, -3] = 1; A[N+1, -2] = -1; 

    
    return A, b, x, y, xc, yc






